<a href="https://colab.research.google.com/github/Jinwooxxi/kagglestudy/blob/main/titanic/Titanic_1st_kernel_%EA%B9%80%EC%A7%84%EC%9A%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
sns.set(font_scale=2.5) 
# 이 두줄은 본 필자가 항상 쓰는 방법입니다. matplotlib 의 기본 scheme 말고 seaborn scheme 을 세팅하고, 
# 일일이 graph 의 font size 를 지정할 필요 없이 seaborn 의 font_scale 을 사용하면 편합니다.
import missingno as msno

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 개요
1. 데이터 셋 확인 - 대부분의 캐글 데이터들은 잘 정제되어 있음. 하지만 null data가 존재. 이를 확인하고, 향후 수정
2. 탐색적 데이터 분석(exploratory data analysis) - 여러 feature 들을 개별적으로 분석하고, feature 들 간의 상관관계를 확인. 여러 시각화 툴을 사용하여 insight를 얻음
3. feature engineering - 모델을 세우기에 앞서, 모델의 성능을 높일 수 있도록 feature 들을 engineering 함. one-hot encoding, class로 나누기, 구간으로 나누기, 텍스트 데이터 처리 등을 함
4. model 만들기 - skearn을 사용해 모델을 만듦. 파이썬에서 머신러닝을 할 때는 sklearn을 사용하면 수많은 알고리즘을 일관된 문법으로 사용할 수 있음. 물론 딥러닝을 위해 tensorflow, pytorch 등을 사용할 수도 있음
5. 모델 학습 및 예척 - trainset을 가지고 모델을 학습 시킨 후, testset을 가지고 prediction 함
6. 모델 평가 -  예측 성능을 우너하는 수준인지 판단. 풀려는 문제에 따라 모델을 평가하는 방식도 달라짐. 학습된 모델이 어떤 것을 학습하였는지 확인

# 1. Dataset 확인


*   파이썬에서 테이블화 된 데이터를 다루는 데 가장 최적화되어 있으며, 많이 쓰이는 라이브러리는 pandas
*   우리는 pandas를 사용하여 데이터셋의 간단한 통계적 분석부터, 복잡한 처리들을 간단한 메소드를 사용
*   파이썬으로 데이터 분석을 한다고 하면 반드시 능숙해져야할 라이브러리, 여러 커널들을 공부하면서 사용법에 익숙해지도록 반복!!
*   캐글 데이터는 대부분 train, test로 나누어져 있음.



In [ ]:
df_train = pd.read_csv("/content/drive/My Drive/titanic/train.csv")
df_test = pd.read_csv("/content/drive/My Drive/titanic/test.csv")

In [ ]:
df_train

* feature : Pclass, Age, SibSp, Parch, Fare
* target label : Survived

In [ ]:
df_train.describe()

In [ ]:
df_test.describe()

## 1.1 Null data check

In [ ]:
for col in df_train.columns:
  msg = "column: {:>10}\t Percent of NaN vlaue: {:.2f}%".format(col, 100*(df_train[col].isnull().sum()/df_train[col].shape[0]))
  print(msg)

In [ ]:
for col in df_test.columns:
  msg = "column: {:>10}\t Percent of NaN vlaue: {:.2f}%".format(col, 100*(df_test[col].isnull().sum()/df_test[col].shape[0]))
  print(msg)

* Train, Test set 둘다 Age(약 20%), Cabin (약 80%), Embarked(Train만 0.22%), null data 존재
* MSNO 라는 라이브러리를 사용하면 null data 존재 더 쉽게 확인 가능

In [ ]:
msno.matrix(df=df_train.iloc[:,:], figsize=(8,8), color=(0.8, 0.5, 0.2));

In [ ]:
msno.bar(df=df_train.iloc[:,:], figsize=(8,8), color=(0.8, 0.5, 0.2));

In [ ]:
msno.bar(df=df_test.iloc[:,:], figsize=(8,8), color=(0.8, 0.5, 0.2));

## 1.2 Target label 확인

* target lable이 어떤 distribution을 가지고 있는지 확인
* 지금과 같은 binary classification 문제의 경우, 1가 0분포가 어떠냐에 따라 모델의 평가 방법이 달라 질 수 있음 

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(18,8))

df_train["Survived"].value_counts().plot.pie(explode=[0,0.1], autopct="%1.1f%%", ax=ax[0], shadow=True)
ax[0].set_title("Pie plot - Survived")
ax[0].set_ylabel("")

sns.countplot("Survived", data=df_train, ax=ax[1])
ax[1].set_title("Count plot - Survived")

plt.show()

* 생존자 38.4%
* target label이 분포가 제법 균일함, 불균일한 경우, 예를 들어서 100중 1이 99, 0이 1개인 경우 만약 모델이 모든것을 1이라 해도 정확도가 99%가 나오게 됨. 0을 찾는 문제라면 이 모델은 원하는 결과를 줄 수 없게 됨.

# 2. EDA(Exploatory data analysis)

* 본격적인 데이터 분석 시작. 적절한 시각화 필요
* 시각화 라이브러리에는 matplotlib, seaborn, plotly 등이 있고 특정 목적에 맞는 소스코드를 정리해 두어 필요할 때마다 참고 하면 편함.

## 2.1 Pcalss

* Pcalss는 ordinal, 서수형 데이터. 카테고리이면서, 순서가 있는 데이터 타입
* Pcalss에 따른 생존률의 차이점 살펴볼 예정. 엑셀 피벗 차트와 유사한 작업. pandas dataframe에서는 groupby를 사용하면 쉽게 할 수 있음. pivot 이라는 메소드도 있음
* "Pclass", "Survived" 를 가져온 후, Pclass로 묶음. 그러고 나면 각 pclass마다 0, 1이 count가 되는데 이를 평균 내면 각 pclass 별 생존률이 나옴
* 아래와 같이 count()를 하게 되면 각 class에 몇명이 있는지 확인할 수 잇으며, sum()을 하면, 216 명중 생존한 (survived=1)사람의 총합을 주게 됨

In [ ]:
df_train[["Pclass", "Survived"]].groupby(["Pclass"], as_index=True).count()

In [ ]:
df_train[["Pclass", "Survived"]].groupby(["Pclass"], as_index=True).sum()

In [ ]:
# crosstab 사용
pd.crosstab(df_train["Pclass"], df_train["Survived"], margins=True).style.background_gradient(cmap="summer_r")

* grouped 객체에 mena()을 하게되면, 각 클래스별 생존률을 얻을 수 있음. 
* class 1이면 80 / (80+136) ≈ 0.63

In [ ]:
df_train[["Pclass", "Survived"]].groupby(["Pclass"], as_index=True).mean().sort_values(by="Survived", ascending=False).plot.bar();

* Pclass가 좋을수록 생존율은 높음
* seaborn의 countplot을 이용하면, 특정 label에 따른 개수를 확인할 수 있음

In [ ]:
y_position = 1.02

f, ax = plt.subplots(1, 2, figsize=(18,8))
df_train["Pclass"].value_counts().plot.bar(color=['#CD7F32','#FFDF00','#D3D3D3'], ax=ax[0])
ax[0].set_title("Number of Passengers By Pclass", y=y_position)
ax[0].set_ylabel("Count")

sns.countplot("Pclass", hue="Survived", data=df_train, ax=ax[1])
ax[1].set_title("Pclass: Survived vs Dead", y=y_position)

plt.show()

* 클래스가 높을수록, 생존확률이 높은걸 확인할 수 있음
* Pclass 1, 2, 3 순서대로 63%, 48%, 25% 
* Pclass가 생존에 가장 큰 영향을 미친다고 생각할 수 있으며, 나중에 모델을 세울 때 Pclass feature를 사용하는 것이 좋을 것이라 판단할 수 있음!

## 2.2 Sex

* 성별로 생존률 확인
* pandas groupby, seaborn countplot을 사용하여 시각화

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(18,8))

df_train[["Sex", "Survived"]].groupby(["Sex"], as_index=True).mean().plot.bar(ax=ax[0])
ax[0].set_title("Survived vs Sex")

sns.countplot("Sex", hue="Survived", data=df_train, ax=ax[1])
ax[1].set_title("Sex: Survived vs Dead")

plt.show()

In [ ]:
df_train[["Sex", "Survived"]].groupby(["Sex"], as_index=True).mean().sort_values(by="Survived", ascending=False)

In [ ]:
# crosstab
pd.crosstab(df_train["Sex"], df_train["Survived"], margins=True).style.background_gradient(cmap="summer_r")

## 2.3 Both Sex and Pclass

* Sex, Pclass 두가지에 관하여 생존이 어떻게 달라지는지 확인
* seaborn의 factorplot을 이용하면, 손쉽게 3개의 차원으로 이루어진 그래프를 그릴수 있음

In [ ]:
sns.factorplot("Pclass", "Survived", hue="Sex", data=df_train, size=6, aspect=1.5);

* 모든 클래스에 female이 살 확률이 male보다 높음
* 또한, 남자, 여자 상관없이 클래스가 높을 수록 생존 확률 높음
* hue 대신 column으로 하면 아래와 같이 나타남

In [ ]:
sns.factorplot(x="Sex", y="Survived", col="Pclass", data=df_train, satureation=.5, size=9, aspect=1);

## 2.4 Age

In [ ]:
print("제일 나이가 많은 탑승객 : {:.1f} Years".format(df_train["Age"].max()))
print("제일 나이가 어린 탑승객 : {:.1f} Years".format(df_train["Age"].min()))
print("탑승객 평균 나이      : {:.1f} Years".format(df_train["Age"].mean()))

* 생존에 따른 Age의 histogram

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12,5))

sns.kdeplot(df_train[df_train["Survived"] == 1]["Age"], ax=ax)
sns.kdeplot(df_train[df_train["Survived"] == 0]["Age"], ax=ax)

plt.legend(["Survived == 1", "Survived == 0"])
plt.show()

* 생존자 중 나이가 어린 경우가 많음

In [ ]:
# Age distribution withing classes
plt.figure(figsize=(10, 6))
df_train["Age"][df_train["Pclass"] == 1].plot(kind="kde")
df_train["Age"][df_train["Pclass"] == 2].plot(kind="kde")
df_train["Age"][df_train["Pclass"] == 3].plot(kind="kde")

plt.xlabel("Age")
plt.title("Age Distribution within classes")
plt.legend(["1st Class", "2nd Class", "3rd Class"])
plt.show()

* class가 높을 수록 나이 많은 사람들의 생존률이 높아짐
* 나이대가 변하면서 생존률이 어떻게 되는지 확인할 예정
* 나이 범위를 점점 넓혀가며, 생존률이 어떻게 되는지 확인

In [ ]:
cummulate_survival_ratio = []

for i in range(0, 80):
  cummulate_survival_ratio.append(df_train[df_train["Age"] < i]["Survived"].sum() / len(df_train[df_train["Age"] < i]["Survived"]))

plt.figure(figsize=(8,8))
plt.plot(cummulate_survival_ratio)
plt.title("Survival rate change depending on range of Age", y=1.02)
plt.ylabel("Survival rage")
plt.xlabel("Range of Age(0~x)")
plt.show()

* 나이가 어릴 수록 생존률이 확실이 높음
* 나이가 중요한 feature로 쓰일 수 있음을 확인!

## 2.5 Pclass, Sex, Age

* 지금까지 본 Sex, Pclass, Age, Survived 모두에 대해 볼 예정, 이를 쉽게 그려주는 것이 violinplot
* x축은 우리가 나눠서 보고싶어하는 case(여기서는 Pclass, Sex), y축은 보고싶어하는 distribution(Age)

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(18,8))

sns.violinplot("Pclass", "Age", hue="Survived", data=df_train, scale="count", split=True, ax=ax[0])
ax[0].set_title("Pclass and Age vs Survived")
ax[0].set_yticks(range(0, 110, 10))

sns.violinplot("Sex", "Age", hue="Survived", data=df_train, scale="count", split=True, ax=ax[1])
ax[1].set_title("Sex and Age vs Survived")
ax[1].set_yticks(range(0, 110, 10))
plt.show()

* 왼쪽 그림은 Pclass 별로 Age의 distribution이 어떻게 다른지, 거기에 생존여부에 따라 구분한 그래프
* 오른쪽 그림도 Sex, 생존에 따른 distribution이 어떻게 다른지 보여주는 그래프
* 생존만 봤을 떄, 모든 클래스에서 나가아 어릴 수록 생존을 많이 한 것을 볼 수 있음
* 오른쪽 그림에서 보면, 명확히 여자가 생존을 많이 한 것을 볼 수 있음
* 여성과 아이를 먼저 챙긴 것을 볼 수 있음

## 2.6 Embarked

* Embarked는 탑승한 항구
* 위에서 해왔던 것과 비슷하게 탑승한 곳에 따른 생존률 확인

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(7,7))
df_train[["Embarked", "Survived"]].groupby(["Embarked"], as_index=True).mean().sort_values(by="Survived", ascending=False).plot.bar(ax=ax);

* 조금의 차이는 있지만 C가 생존률이 가장 높음
* 모델에 얼마나 큰 영향을 미칠지는 모르겠지만, 그래도 사용
* 모델을 만들고 나면 우리가 사용한 feature들이 얼마나 중요한 역할 을 했는지 확인해볼 수 있음. 이는 추후에 모델을 만들고 난 다음에 살펴볼 예정
* 다른 feature로 split하여 살펴볼 예정

In [ ]:
f, ax = plt.subplots(2, 2, figsize=(20,15))

sns.countplot("Embarked", data=df_train, ax=ax[0,0])
ax[0,0].set_title("(1) No. Of Passengers Boarded")

sns.countplot("Embarked", hue="Sex", data=df_train, ax=ax[0,1])
ax[0,1].set_title("(2) Male-Female Split for Embarked")

sns.countplot("Embarked", hue="Survived", data=df_train, ax=ax[1,0])
ax[1,0].set_title("(3) Embarked vs Survived")

sns.countplot("Embarked", hue="Pclass", data=df_train, ax=ax[1,1])
ax[1,1].set_title("(4) Embarked vs Pclass")

plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

* Figure(1) - 전체적으로 봤을 때, S에거 가장 많은 사람이 탑승
* Figure(2) - C와 Q는 남여 비율이 비슷하고, S는 남자가 더 많음
* Figure(3) - 생존확률이 S의 경우 많이 낮은 것을 볼 수 있음
* Figure(4) - Class로 split 해서 보니, C가 생존확률이 높은건 클래스가 높은 사람이 많이 타서 그런 것을 확인, S는 3rd class가 많이서 생존확률이 낮게 나옴

## 2.7 Family - SibSP(형제, 자매) + Parch(부모, 자녀)

* SibSp와 Parch를 합하면 Famlily

In [ ]:
df_train["FamilySize"] = df_train["SibSp"] + df_train["Parch"] + 1 # 자신을 포함해야하니 1을 더함
df_test["FamilySize"] = df_test["SibSp"] + df_test["Parch"] + 1 # 자신을 포함해야하니 1을 더함

In [ ]:
print("Maximum size of Family: ", df_train["FamilySize"].max())
print("Minimum size of Family: ", df_train["FamilySize"].min())

* FamilySize와 생존의 관계

In [ ]:
f, ax = plt.subplots(1, 3, figsize=(40,10))

sns.countplot("FamilySize", data=df_train, ax=ax[0])
ax[0].set_title("(1) No. Of Passengers Boarded", y=1.02)

sns.countplot("FamilySize", hue="Survived", data=df_train, ax=ax[1])
ax[1].set_title("(2) Survived countplot depending on FamilySize", y=1.02)

df_train[["FamilySize", "Survived"]].groupby(["FamilySize"], as_index=True).mean().sort_values(by="Survived", ascending=False).plot.bar(ax=ax[2])
ax[2].set_title("(3) Survived rate depending on Family Size", y=1.02)

plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

* Figure (1) - 가족크기가 1~11까지 있음을 볼수 있음, 대부붙 1명이고 그 다음으로 2, 3, 4명 순
* Figure (2), (3) - 가족크기에 따른 생존자 비교, 가족이 4명인 경우 가장 생존확률이 높음, 가족수가 많아질수록 (5,6,7,8,11) 생족확률이 낮아짐, 가족수가 너무 작거나 크면 생족확률이 작아짐. 3~4명이 생존확률이 높은 것을 확인할 수 있음

## 2.8 Fare

* Fare는 탑승요금, contious feature → histogram

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(8,8))

g = sns.distplot(df_train["Fare"], color="b", label="Skewness : {:.2f}".format(df_train["Fare"].skew()), ax=ax)
g = g.legend(loc="best")

* distribution이 매우 비대칭적(high skewness), 만약 이대로 모델에 넣어준다면 모델이 잘못 학습할 수 있음, 몇개 없는 outlier에 대해서 너무 민감하게 반응한다면, 실제 예측 시에 좋지 못한 결과를 부를 수 있음
* outlier의 영향을 줄이기 위해 Fare에 log를 취할 예정
* pandas의 유용한 기능 사용
* → dataframe의 특정 columns에 공통된 작업(함수)를 적용하고 싶으면 아래의 map, apply를 사용하면 쉽게 적용 가능
### ※ 매우 유용한 기능이므로 필히 숙지!! 
* 지금 원하는 것은 Fare columns의 데이터 모두를 log 값 취하는 것, 파이썬의 간단한 lambda 함수를 이용해 간단한 로그를 적용하는 함수를 map에 인수로 넣어주면 Fare columns데이터에 그대로 적용이 됨.


In [ ]:
df_test.loc[df_test.Fare.isnull(), "Fare"] = df_test["Fare"].mean() # test set에 있는 nan value를 평균값으로 치환

df_train["Fare"] = df_train["Fare"].map(lambda i: np.log(i) if i>0 else 0)
df_test["Fare"] = df_test["Fare"].map(lambda i: np.log(i) if i>0 else 0)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,8))
g = sns.distplot(df_train["Fare"], color="b", label="Skewness {:.2f}".format(df_train["Fare"].skew()), ax=ax)
g = g.legend(loc="Best")

* log를 취하니 비대칭이 사라짐
* 이런 작업을 통해 모델이 좀 더 좋은 성능을 내도록 할 수 있음
* feature engineering에 들어가는 부분이지만 지금 해봄
* 모델을 학습시키기 위해, 성능향상을 위해 feature 들에 여러 조작을 가하거나, 새로운 feature를 추가하는 것이 feature engineering이라고 함

## 2.9 Carbin

* NaN이 대략 80%이므로, 생존에 영향을 미칠 중요한 정보를 얻어내기가 쉽지 않음 → 모델에 포함시키지 않는다

In [ ]:
df_train.head()

## 2.10 Ticket - 추후 분석

* string data

In [ ]:
df_train['Ticket'].value_counts()

* 매우 다양함 → 어떤 특징을 이끌어 내어 생존과 연결시킬 지 고민해봐야함
* 일단 여기서는 넘기고 모델 성능을 향상 시키기 위해 검토해보는 것으로 ...

# 3. Feature engineering

* 본격적인 feature engineering
* 가장 먼저 data set에 존재하는 null data 채움
* null data를 포함하는 feature의 statistics를 참고하거나, 다른 아이디어를 통해 채울 수 있음
* null data를 어떻게 채우냐에 따라 모델의 성능이 좌지우지될 수 있기 때문에 **신경써야함**
* Featrue engineering은 실제 모델의 학습에 쓰려고 하는 것이므로 train, test 둘다 적용!!

## 3.1 Fill Null

### 3.1.1 Fill null in Age using title

* Age에는 null data 가 177개 존재, 이를 채울 아이디어 중 title + statistics를 사용
* 영어에서는 Miss, Mrr, Mrs 같은 title이 존재, 각 탑승객 이름에 title이 존재하므로 이를 사용
* pandas series에는 data를 string으로 바꿔주는 str method, 거기에는 정규식을 적용하게 해주는 extract method가 있음, 이를 사용하여 title을 쉽게 추출할 수 있음, title을 Initial column에 저장 할 예정

In [ ]:
df_train["Initial"] = df_train.Name.str.extract("([A-Za-z]+)\.") # lets extract the Salutation
df_test["Initial"] = df_test.Name.str.extract("([A-Za-z]+)\.") # lets extract the Salutation

* pandas의 crosstab을 이용하여 추출한 Initial과 Sex 간의 count 확인

In [ ]:
pd.crosstab(df_train["Initial"], df_train["Sex"]).T.style.background_gradient(cmap="summer_r") # Checking the Initials with the Sex

* 위 table을 참고하여 남자, 여자가 쓰는 initail을 구분 → replace 메소드를 사용하면, 특정 데이터 값을 원하는 값으로 치환

In [ ]:
df_train['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don', 'Dona'],
                            ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr', 'Mr'],inplace=True)

df_test['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don', 'Dona'],
                           ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr', 'Mr'],inplace=True)

In [ ]:
df_train.groupby("Initial").mean()

* 여성과 관계있는 Miss, Mr, Mrs가 생존확률이 높은 것


In [ ]:
df_train.groupby("Initial")["Survived"].mean().plot.bar();

* null data를 채우는 방법 다양함. statistics를 활용하는 방법, null data가 ㅇ벗는 데이터를 기반으로 새로운 머신러닝 알고리즘을 ㅁ나들어 예측해서 채워넣는 방식 등 다양함. 여기서는 ***statistics를 활용한 방법을 사용***
* statistics는 여기서 train data의 것을 의미.
* test를 unseen으로 둔 상태로 놔둬야 하며, train에서 얻은 statistics를 기반으로 test의 null data를 채워줘야 함!

In [ ]:
df_train.groupby("Initial").mean()

* Age의 평균을 이용해 Null value 채울 예정
* pandas dataframe을 다룰 때에는 boolean array를 이용해 indexing하는 방법이 편리함!
* 아래 코드 첫출을 해적하자면 isnull()이면서 Initial이 Mr인 조건을 만족하는 row(탑승객)의 "Age"의 값을 33으로 치환한다는 의미
#### ※ 매우 유용한 기능이므로 필히 숙지!!
****loc + boolean + column을 사용해 값을 치환하는 방법***

In [ ]:
df_train.loc[(df_train.Age.isnull())&(df_train.Initial=="Mr"), "Age"] = 33
df_train.loc[(df_train.Age.isnull())&(df_train.Initial=="Master"), "Age"] = 5
df_train.loc[(df_train.Age.isnull())&(df_train.Initial=="Miss"), "Age"] = 22
df_train.loc[(df_train.Age.isnull())&(df_train.Initial=="Mrs"), "Age"] = 36
df_train.loc[(df_train.Age.isnull())&(df_train.Initial=="Other"), "Age"] = 46

df_test.loc[(df_test.Age.isnull())&(df_test.Initial=="Mr"), "Age"] = 33
df_test.loc[(df_test.Age.isnull())&(df_test.Initial=="Master"), "Age"] = 5
df_test.loc[(df_test.Age.isnull())&(df_test.Initial=="Miss"), "Age"] = 22
df_test.loc[(df_test.Age.isnull())&(df_test.Initial=="Mrs"), "Age"] = 36
df_test.loc[(df_test.Age.isnull())&(df_test.Initial=="Other"), "Age"] = 46

* 간단하게 Null을 채웠지만 다양한 방법이 존재 
* 참고자료 : https://www.kaggle.com/yassineghouzam/titanic-top-4-with-ensemble-modeling

### 3.1.2 Fill Null in Embarked

In [ ]:
print("Embarked has", sum(df_train["Embarked"].isna()), "Null values")

* Embarked는 Null value가 2개, S에가 가장 많은 탑승 인원이 있었으므로 S로 간단하게 채움
* dataframe의 fillna method를 이용하면 쉽게 채울 수 있음
* inplace=True로 하면 실제 적용

In [ ]:
df_train["Embarked"].fillna("S", inplace=True)

## 3.2 Change Age(continuos to categoricla)

* Age는 현재 continuous feature, 이대로 써도 모델을 세울 수 있지만 Age를 몇개의 group으로 나누어 category화 시켜 줄수도 있음. continuous를 categorical로 바꿔주면 information loss가 생길수 있지만 이번에는 다양한 방법으로 진행하는 것이 목적이기 때문에 그래도 진행할 예정

* 방법은 여러가지, dataframe의 indexing 방법인 loc를 사용하여 직접해줄 수 있고, apply를 사용해 함수를 넣어줄 수 있음
### ※ 매우 요용한 기능이므로 필히 숙지!!
* loc, loc는 자주 쓰게 되므로 사용법 숙지하면 좋음!!
* 나이는 10살 간격으로 나눔

In [ ]:
df_train["Age_cat"] = 0
df_train.loc[df_train["Age"] < 10, "Age_cat"] = 0
df_train.loc[(10 <= df_train["Age"]) & (df_train["Age"] < 20), "Age_cat"] = 1
df_train.loc[(20 <= df_train["Age"]) & (df_train["Age"] < 30), "Age_cat"] = 2
df_train.loc[(30 <= df_train["Age"]) & (df_train["Age"] < 40), "Age_cat"] = 3
df_train.loc[(40 <= df_train["Age"]) & (df_train["Age"] < 50), "Age_cat"] = 4
df_train.loc[(50 <= df_train["Age"]) & (df_train["Age"] < 60), "Age_cat"] = 5
df_train.loc[(60 <= df_train["Age"]) & (df_train["Age"] < 70), "Age_cat"] = 6
df_train.loc[70 <= df_train["Age"], "Age_cat"] = 7

df_test["Age_cat"] = 0
df_test.loc[df_test["Age"] < 10, "Age_cat"] = 0
df_test.loc[(10 <= df_test["Age"]) & (df_test["Age"] < 20), "Age_cat"] = 1
df_test.loc[(20 <= df_test["Age"]) & (df_test["Age"] < 30), "Age_cat"] = 2
df_test.loc[(30 <= df_test["Age"]) & (df_test["Age"] < 40), "Age_cat"] = 3
df_test.loc[(40 <= df_test["Age"]) & (df_test["Age"] < 50), "Age_cat"] = 4
df_test.loc[(50 <= df_test["Age"]) & (df_test["Age"] < 60), "Age_cat"] = 5
df_test.loc[(60 <= df_test["Age"]) & (df_test["Age"] < 70), "Age_cat"] = 6
df_test.loc[70 <= df_test["Age"], "Age_cat"] = 7

In [ ]:
def category_age(x):
  if x < 10:
    return 0 
  elif x < 20:
    return 1
  elif x < 30:
    return 2
  elif x < 40:
    return 3
  elif x < 50:
    return 4
  elif x < 60:
    return 5
  elif x < 70:
    return 6
  else:
    return 7

df_train["Age_cat_2"] = df_train["Age"].apply(category_age)

* 두가지 방법이 잘 적용되었다면 둘다 같은 결과 !
* 이를 확인하기 위해 Series간 boolean 비교 후 all() 메소드를 사용.
* all() 메소드는 모든 값이 True 면 True, 하나라도 False가 있으면 False

In [ ]:
print("1번 방법, 2번 방법 둘다 같은 결과를 내면 True 줘야함 →", (df_train["Age_cat"] == df_train["Age_cat_2"]).all())

* True!!
* 중복되는 Age_cat 컬럼괌 원래 컬럼 Age 제거

In [ ]:
df_train.drop(["Age", "Age_cat_2"], axis=1, inplace=True)
df_test.drop(["Age"], axis=1, inplace=True)

## 3.3 Change Initial, Embarked and Sex (string to numerical)

* 현재 Initial은 Mr, Mrs, Miss, Master, Other 총 5개로
* 카테고리로 표현되어 있는 데이터를 모델에 입력시키기 위해 컴퓨터가 인식할수 있도록 수치화 시켜줘야함

* map method를 가지고 간단히 할 수 있음
* 사전 순서대로 정리하여 mapping

In [ ]:
df_train["Initial"] = df_train["Initial"].map({"Master":0, "Miss":1, "Mr":2, "Mrs":3, "Other":4})
df_test["Initial"] = df_test["Initial"].map({"Master":0, "Miss":1, "Mr":2, "Mrs":3, "Other":4})

* Embarked에도 C, Q, S로 이루어져 있음, map을 이용해 변경
* 특정 컬럼에 어떤 값들이 있는지 확인, unique(), value_counts()

In [ ]:
df_train["Embarked"].unique()

In [ ]:
df_train["Embarked"].value_counts()

In [ ]:
df_train["Embarked"] = df_train["Embarked"].map({"C":0, "Q":1, "S":2})
df_test["Embarked"] = df_test["Embarked"].map({"C":0, "Q":1, "S":2})

* Null 값 확인
* Series 값들이 null인지 아닌지 확인 → any()

In [ ]:
df_train["Embarked"].isna().any()

In [ ]:
df_train['Sex'] = df_train['Sex'].map({'female': 0, 'male': 1})
df_test['Sex'] = df_test['Sex'].map({'female': 0, 'male': 1})

* 각 feature 간의 상관관계 확인
* 두 변수간의 Pearson corrlation을 구하면 (-1, 1)사이의 값을 얻을 수 있음
* -1로 갈수록 음의 상관관계, 1로 갈수록 양의 상관관계 의미, 0은 상관관계가 없다는 의미
* rxy=Cov(x,y)SxSy=1n−1∑ni=1(xi−x¯)(yi−y¯)SxSy
* 여러 feature를 matrix형태로 확인 → heatmap plot

In [ ]:
heatmap_data = df_train[["Survived", "Pclass", "Sex", "Fare", "Embarked", "FamilySize", "Initial", "Age_cat"]]

colormap = plt.cm.RdBu
plt.figure(figsize=(14,12))
plt.title("Pearson Correlation of Features", y=1.05, size=15)
sns.heatmap(heatmap_data.astype(float).corr(), linewidths=0.1, vmax=1.0, square=True, cmap=colormap,
            linecolor="white", annot=True, annot_kws={"size":16})

del heatmap_data

* EDA에서 살펴봤듯이, Sex, Pclass가 Survived에 상관관계가 어느정도 있음을 볼 수 있음
* 생각보다 fare와 Embarked도 상관관계가 있음을 알 수 있음
* 여기서 얻을 수 있는 정보는 서로 강한 상관관계를 가지는 feature들이 없다는 점
* 모델을 학습시킬 때, 불필요한(redundant, superfluous) feature 가 없다는 것을 의미, 1 또는 -1 의 상관관계를 가진 feature A, B 가 있다면, 우리가 얻을 수 있는 정보는 사실 하나
* 실제 모델을 학습시키기 앞서서 data preprocessing(전처리) 진행!

## 3.4 One-Hot encoding on Initial and Embarked

* 수치화시킨 카테고리 데이터를 그대로 넣어도 되지만, 모델의 성능을 높이기 위해 one-hot encoding을 할 수 있음
* 수치화는 간단히 Master == 0, Miss == 1, Mr == 2, Mrs == 3, Other == 4 로 매핑해주는 것
* One-hot encoding 은 위 카테고리를 아래와 같이 (0, 1) 로 이루어진 5차원의 벡터로 나타내는 것
* pandas의 get_dummies를 사용하면 쉽게 해결할 수 있음
* 총 5개의 카테고리, one-hot encoding을 하고 나면 5개 컬럼 생성
* Initial을 prefix로 두어서 구분이 쉽게 만들어 줌

In [ ]:
df_train = pd.get_dummies(df_train, columns=["Initial"], prefix="Initial")
df_test = pd.get_dummies(df_test, columns=["Initial"], prefix="Initial")

In [ ]:
df_train.head()

In [ ]:
df_train = pd.get_dummies(df_train, columns=['Embarked'], prefix='Embarked')
df_test = pd.get_dummies(df_test, columns=['Embarked'], prefix='Embarked')

* sklearn의 Labelencoder + OnHotencoder 이용해도 가능
* 가끔 카테고리가 100개가 넘는 경우도 있는데 이때는 다른 방법 사용

## 3.5 Drop Columns

In [ ]:
df_train.drop(['PassengerId', 'Name', 'SibSp', 'Parch', 'Ticket', 'Cabin'], axis=1, inplace=True)
df_test.drop(['PassengerId', 'Name',  'SibSp', 'Parch', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [ ]:
df_train.head()

In [ ]:
df_train.head()

# 4. Building machin learing model and prediction using the traind model

In [ ]:
# importing all the required ML packages
from sklearn.ensemble import RandomForestClassifier # 유명한 randomforestclassfier
from sklearn import metrics # 모델의 평가
from sklearn.model_selection import train_test_split # traning set을 쉽게 나눠주는 함수

### 매우 중요!! 숙지 필수 !!!
* Sklearn은 머신러닝 처음부터 끝까지 다 있음
* feature engineering, preprocessing, 지도학습 알고리즘, 비지도학습 알고리즘, 모델평가, 파이프라인 등 머신러닝에 관련된 모든 작업들이 손쉬운 인터페이스로 구현, 데이터 분석 + 머신러닝을 하고 싶다면 반드시 숙지!!

* 파이썬 라이브러리를 활용한 머신러닝(Introduction to machine larning with Python)책을 사서 공부하시길 매우 추천

* 타이타익 문제는 target class(Survived)가 있으면, target class는 0,1로 이루어져 있으므로 binary classification 문제
* train set의 survived를 제외한 input을 가지고 모델을 최적화 시켜서 각 샘플(탑승객)의 생존유무를 판단하는 모델을 만들어 냄
* 그 후 모델이 학습하지 않았던 test set을 input으로 주어서 test set의 각 샘플(탑승객)의 생존 유무를 예측

## 4.1 Preparation - Split dataset into train, valid, test set

* 가장 먼저 학습 데이터, target label 분리

In [ ]:
X_train = df_train.drop("Survived", axis=1).values
target_label = df_train["Survived"].values
X_test = df_test.values

* train, test 뿐만 아니라 좋은 모델을 만들기 위해 valid set 추가로 생성
* train_test_split 사용

In [ ]:
X_tr, X_vld, y_tr, y_vld = train_test_split(X_train, target_label, test_size=0.3, random_state=2018)

* sklearn 에서는 여러 머신러닝 알고리즘을 지원
* 참조 : http://scikit-learn.org/stable/supervised_learning.html#supervised-learning
* 랜덤포레스트 모델 사용
* 랜덤포레스트는 결정트리기반 모델, 여러 결정 트리들을 앙상블한 모델
* 알고리즘에는 여러 파라미터들 존재. 랜덤포레스트분류기도 n_estimators, max_features, max_depth, min_samples_split, min_samples_leaf 등 여러 파라미터들이 존재. 이것들이 어떻게 세팅되냐에 따라 같은 데이터셋이라 하더라도 모델의 성능이 달라짐.
* 파라미터 튜닝은 시간, 경험, 알고리즘에 대한 이해 등이 필요. 결국 많이 써봐야 모델도 잘 세울 수 있음. 여러 데이터셋을 가지고 모델을 이리저리 써봐야 튜닝하는 감이 생길 수 있음
* 모델 객체를 만들고, fit 메소드로 학습
* 그런 후 valid set input 을 넣어주어 예측값(X_vld sample(탑승객)의 생존여부)를 얻움

## 4.2 Model generation and prediction

In [ ]:
model = RandomForestClassifier()
model.fit(X_tr, y_tr)
prediction = model.predict(X_vld)

* 모델 생성 및 예측까지 

In [ ]:
print("총 {}명 중 {:.2f}% 정확도로 생존을 맞춤".format(y_vld.shape[0], 100*metrics.accuracy_score(prediction, y_vld)))

## 4.3 Feature importance

* 학습된 모델을 feature importance를 가짐, 방금 만든 모델이 어떤 feature에 영향을 많이 받았는지 확인할 수 있음

* 쉽게 말해 10 = 4x1 + 2x2 + 1x3을 생각하면 x1이 결과값(10)에 큰영향을 준다고 생각할 수 있음. feature importance는 4, 2, 1을 이야기하며, x1이 가장 큰 값(4)를 가지므로, 이 모델에 가장 큰영향을 미친다고 할 수 있음

* 학습된 모델은 기본적으로 featureimportance를 가지고 있어서 쉽게 수치를 얻을 수 있음
* pandas series를 이요하면 쉽게 sorting을 하여 그래프를 그릴 수 있음

In [ ]:
from pandas import Series

featrue_importance = model.feature_importances_
Series_feat_imp = Series(featrue_importance, index=df_test.columns)

In [ ]:
plt.figure(figsize=(8,8))
Series_feat_imp.sort_values(ascending=True).plot.barh()

plt.xlabel("Feature importance")
plt.ylabel("Feature")
plt.show()

* Fare가 가장 큰 영향력, Age_cat, Initial_2, Sex, Pclass 순
* feature importance를 보고 실제로 Fare가 가장 중요한 feature로 판단 내릴 수 있지만, 결국 모델에 귀속되는 하나의 결론이므로 통계적으로 좀 더 살펴보긴 해야함
* feature importance를 가지고 좀 더 정확도가 높은 모델을 얻기 위해 feature selection을 할 수 있고, 좀 더 빠른 모델을 위해 feature 제거를 할 수 있음

## 4.4 Prediction on Test set

* test set을 모델이 주어 생존여부 예측

In [ ]:
submission = pd.read_csv("/content/drive/My Drive/titanic/gender_submission.csv")

In [ ]:
submission.head()

In [ ]:
prediction = model.predict(X_test)
submission['Survived'] = prediction